# Readprot 1.0
Extrahiert aus den PDF-Protokollen im Verzeichnis '2018' alle Voten und ordnet sie dem/der jeweiligen RednerIn zu.
Das Ergebnis-Dataframe wird gespeichert für die Auswertung.

## 1 - Libraries, Variablen und Funktionen

### 1.1 Import
Bennötigt werden regex, pandas, json, requests, pickle, glob, difflib, datetime, os.path und pdfminer. 

In [ ]:
import re #regex
import pandas as pd
import json
import requests
import pickle
import glob
import difflib
from datetime import datetime
from os.path import exists

import pdfminer
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from pdfminer.layout import LTChar
from pdfminer.converter import PDFPageAggregator

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 1.2 Variablen

In [ ]:
#Variablen zur Nutzung des Gemeinderats-API    
endpunkt_suche = 'http://www.gemeinderat-zuerich.ch/api/Mitglieder?'
endpunkt_abfrage = 'http://www.gemeinderat-zuerich.ch/api/Mitglieder/details?'

#Erstellt eine Liste der Protokoll-Files, die eingelesen werden sollen            
list_of_pdfs = sorted(glob.glob('2018/*.pdf'))

### 1.3.1 Hilfsfunktionen
verschiedene kleinere Funktionen, die später benötigt werden

In [ ]:
# Load/Save uber_df, nampi_dict and corr_dict
def uber_df_load():
    with open('uber_df.pickle', 'rb') as f:
        return pickle.load(f)

def nampi_dict_load():
    with open('nampi_dict.pickle', 'rb') as f:
        return pickle.load(f)
        
def uber_df_save():
    with open('uber_df.pickle', 'wb') as f:
        pickle.dump(uber_df, f, pickle.HIGHEST_PROTOCOL)    

def nampi_dict_save():
    with open('nampi_dict.pickle', 'wb') as f:
        pickle.dump(nampi_dict, f, pickle.HIGHEST_PROTOCOL)
        
#Funktion, welche die Voten jeder einzelnen Sitzung zu Kontrollzwecken in ein File speichert
def save_votelist(votelist):
    with open('votelists/'+sitzungs_id+'.pickle', 'wb') as f:
              pickle.dump(votelist, f, pickle.HIGHEST_PROTOCOL)

### 1.3.2 Kernfunktionen
Funktion für das Einlesen der Protokolle sowie für die Überprüfung der Namen der RednerInnen

In [ ]:
# Funktion, welche die PDF files mit pdfminer einliest und RednerInnen und Voten extrahiert und erste Schritte zur Bereinigung vornimmt
# Code: ChatGPT
def extract_pdf_text(file_name):
      
    print(file_name)
   
    # Open the PDF file
    fp = open(file_name, 'rb')

    # Create a PDF parser object associated with the file object
    parser = PDFParser(fp)

    # Create a PDF document object that stores the document structure
    document = PDFDocument(parser)

    # Create a PDF resource manager object that stores shared resources
    rsrcmgr = PDFResourceManager()

    # Set parameters for analysis
    laparams = LAParams()

    # Create a PDF page aggregator object
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Save the output of the code to a list
    output = []
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        layout = device.get_result()
        for obj in layout:
            if hasattr(obj, "get_text"):
                for line in obj:
                    for char in line:
                        if isinstance(char, LTChar):
                            output.append((char.get_text(), char.fontname))

    # Process the output word by word
    bold_word = ""
    italic_word = ""
    nested_list = []
    previous_font = ""

    for char, font in output:
        if "Arial-BoldItalicMT" in font:
            if "Arial-ItalicMT" in previous_font:
                nested_list.append([bold_word, italic_word])
                bold_word = ""
                italic_word = ""
            bold_word += char
            previous_font = font
        elif "Arial-ItalicMT" in font:
            italic_word += char
            previous_font = font
        else:
            if bold_word:
                nested_list.append([bold_word, italic_word])
                bold_word = ""
                italic_word = ""

    # Check if there's any remaining text that needs to be added to the list
    if bold_word:
        nested_list.append([bold_word, italic_word])

    # If the first element of a sublist is empty, but the second element isnt, add that to the second element of the previous sublist
    final_list = []
    for i in range(len(nested_list)):
        stripped_first_elem = nested_list[i][0].strip()
        if stripped_first_elem.startswith("ST") or stripped_first_elem.startswith("Ratspräsident") or stripped_first_elem.startswith('Pierre Heusser') or stripped_first_elem.startswith("Vizepräsident") or stripped_first_elem.startswith('Pierre Heusser') or any(char.isdigit() for char in stripped_first_elem) == True:
            continue
            
        elif nested_list[i][0].strip() == "" and nested_list[i][1].strip() != "":
            if final_list:
                final_list[-1][1] += nested_list[i][1]
        
        elif nested_list[i][0].strip() == ":" and nested_list[i][1].strip() != "":
            if final_list:
                final_list[-1][1] += nested_list[i][1] 
                
        elif nested_list[i][0].strip() == ".":
            if final_list:
                final_list[-1][1] += nested_list[i][1]
    
        elif nested_list[i][0].strip() == ".":
            if final_list:
                final_list[-1][1] += nested_list[i][1]
        
        elif nested_list[i][0].strip() != "" and nested_list[i][0][:2] != "ST":

            final_list.append(nested_list[i])
        
    # Delete spaces at the beginning of the second element of every sublist
    for i in range(len(final_list)):
        final_list[i][1] = final_list[i][1].lstrip()
        
    # Delete "Dr." ...
    for i in range(len(final_list)):
        final_list[i][0] = final_list[i][0].replace('Dr.', '')    
        
    final_list = [sublist for sublist in final_list if len(sublist[0]) >= 10]       

    return final_list

In [ ]:
# Lade Liste mit korrekten Namen und Parteien
with open("list_file.txt") as f:
    name_list = f.read().splitlines()

# Funktion,die Namen und Parteien der RednerInnen überprüft und falls nötig korrigiert
# Input name = Vorname Name (Partei)
# Output: korrekter Name und Partei Vorname Name (Partei)
# Code: ChatGPT
threshold=0.72
def check_name_in_list(name):    
    name = name.lstrip()
    name = name.split(")")[0] + ")"
    if name in name_list:
        return name
    else:
        closest_match = difflib.get_close_matches(name, name_list, n=1, cutoff=threshold)
        if closest_match:
            if threshold < 0.95:               
                return closest_match[0]
            else:                
                return "Incorrect"
        else:
            return "Incorrect"

### 1.3.4 - Funktionen für Aufbau des dataframes

In [ ]:
#Funktion, die aus der vorangehenden nested list mit den Voten ein erstes dataframe erstellt
def build_initial_df (votelist):
    df = pd.DataFrame(votelist)
    return df

#Funktion, welches die Textlängen der Voten zum df hinzufügt
def add_length (intitial_df):   
    initial_df['Länge'] = initial_df[1].apply(lambda x: len (x)) # Len ermitteln und neue Spalte bilden
    return initial_df    

#Funktion, welche die noch in den Namen enthaltenen Parteien absplittet und in eine eigene Spalte im df schiebt. Bereinigung einzelner Sonderfälle. 
def split_parties (df_with_length):
    parteien = []
    for values in df_with_length[0]:
        #if values == 'David Garcia Nuñez':
            #values = 'David Garcia Nuñez (AL)'
        #elif values == 'Walter Anken':
            #values = 'Walter Anken (SVP)'
        #elif values == 'Sven Sobernheim':
            #values = 'Sven Sobernheim (GLP)'               
        
        parteien.append(re.search(r'(?<=\()(.+?)(?=\))', values).group())
    
    df_with_length['Partei'] = parteien
    
    for values in df_with_length[0]:
        if '(' in values and ')' in values:
            partei = (re.search(r'(?<=\()(.+?)(?=\))', values))[1]
            name_neu = values.replace('('+partei+')', '')
            name_neu = name_neu.replace(':','')
            name_neu = name_neu.strip()
            name_ohne_partei.append(name_neu)
        else:
            name_neu = values.strip()
            name_ohne_partei.append(name_neu)
        
    df_with_length[0] = name_ohne_partei
    df_with_length[0] = df_with_length[0].str.strip()
    
    return df_with_length

### 1.3.5 Funktionen für Abfrage des Gemeinderats-API

In [ ]:
#Funktion, welche die Namen der RednerInnen aus dem df für die API-Abfrage aufbereitet
def build_searchlist (df_with_corrnames):
    search_list = []
    names = df_with_corrnames[0].values.tolist()
    names = [x.replace(' ','+') for x in names] # Space durch + ersetzen
    parties = df_with_corrnames['Partei'].values.tolist()
    search_list.extend([list(a) for a in zip(names, parties)])
    return search_list

# Suchabfrage nach Name
# Das API ignoriert exakte Namen und Parteizugehörigtkeit und kann deshalbe falsche Treffer zurückgeben.
def membersearch(name, partei):    
    name = name.strip()
    #if 'M ichael Schmid' in name:
        #name.replace('M ichael Schmid', 'Michael Schmid')
    pre_filtered = []
    req = endpunkt_suche+'name='+name+'&parteiId='+partei+'&includeInactive=true'
    r_suche = requests.get(req)
    r_suche_out = r_suche.json()
    for every in r_suche_out:
        if every['Partei'] == partei:
            pre_filtered.append(every)
    
    return pre_filtered    

# Funktion die Suchresultate validiert  - welcher Name stimmt wirklich? 
# Es wird überprüft, ob Name und Partei des API-Resultats tatsächlich dem Input entspricht
# Ausgegeben wird die MID des API, welches jedes Ratsmitglied eindeutig identifziert
def memberselect (searchresult):
    for every_result in searchresult:        
        resname = every_result['Vorname']+' '+every_result['Name']        
        snamrep = searchname.replace('+',' ')        
        if resname == snamrep:        
            corrid = every_result['Id']
            return corrid

# Funktion, die Mitgliederdetails anhand der ermittelten korrekten MID abfragt
# Ausgeben wird eine Liste mit Anrede, Vorname, Name, Partei, Geburtstag, MID, sowie (zur Kontrolle) Name und Partei gemäss dataframe mit den Voten
def memberget (mid):
    type(mid)
    url = endpunkt_abfrage+'mid='+mid
    r = requests.get(url)
    out = r.json()
    vorname = out['Vorname']
    name = out['Name']
    partei = out['Partei']
    nampi = vorname+' ' +name+partei
    put = [out['Anrede'], out['Vorname'], out['Name'], out['Partei'], out['Geburtstag'], out['Id'], nampi]
    
    return put

# Funktion, welche eine lokale Datenbank mit bereits früher abgefragten Mitgliederdetails abfragt. 
def query_nampi_dict (name,partei):
    nampi = name+partei
    output = nampi_dict[nampi]
    return output

# Funktion, welche einzelne Schritte/Funktionen zur Abfrage von Mitgliederdetails bündelt (check_nampi, quey_nampi, membersearch, memberselect und memberget)
# Ausgegeben Ausgeben wird eine Liste [Anrede, Vorname, Name, Partei, Geburtstag, MID, Name und Partei gemäss dataframe mit den Voten]
# Details zu jedem Ratsmitglied werden laufend in eine lokale Datenbank gespeichert
# Die lokale Datenbank wird bei der Abfrage bevorzugt, da das API sehr träge ist. So wird jedes Mitglied nur ein einziges Mal über das API abgefragt.
def membquest(name, partei):
    name = name.replace(':', '')
    name = name.strip()  
    if partei == 'FPD':
        partei = 'FDP'
    nampi = name+partei
    if nampi in nampi_dict:
        put = query_nampi_dict (name, partei)
        
    else:
        searchres = membersearch (name, partei)
        midn = memberselect (searchres)    
        put = memberget (midn)
        nampi = name+partei
        nampi_dict[nampi]=put
    return put

In [ ]:
#Funktion, welche das bereits vorhandene df mit Voten, RednerInne, Längen, etc. aufräumt
def clean_final_df (final_df):
    final_df = final_df.iloc[:, [4,0,3,5,1,2,6,7]] # Reihenfolge der columns
    final_df.columns = ['Anrede', 'Name', 'Partei', 'Geburtsdatum', 'Votum', 'Länge', 'Id', 'nampi'] # Beschriftung columns
    final_df['Geburtsdatum'] =  pd.to_datetime(final_df['Geburtsdatum'], infer_datetime_format=True) # Geburtsdatum richtig setzen
    return final_df               


## 2 Code der die Funktionen ausführt

### 2.1 load/init

In [ ]:
# Lade (falls vorhanden) oder initialisiere uber_df, das df mit den Endresultaten
if exists('uber_df.pickle') == True:
    uber_df = uber_df_load()
else:
    uber_df = pd.DataFrame(columns=['Anrede','Name','Partei','Geburtsdatum','Sitzung','Länge','Id', 'nampi'])

# Lade oder initialisere nampi_dict, die lokale Datenbank mit Details zu den Ratsmitgliedern
if exists('nampi_dict.pickle') == True:
    nampi_dict = nampi_dict_load()
else:
    nampi_dict = {} # key=nampi value=list'Anrede', 'Vorname', 'Name', 'Partei', 'Geburtstag', 'Id'

### 2.2 Bündeln von Funktionen

In [ ]:
# Ausgegeben wird eine Liste von RednerInnen, deren Details abgefragt werden müssen sowie (mittels 'global') ein df mit den Voten und RednerInnen
def part1 (every_pdf):
    global sitzungs_id
    sitzungs_id = ''.join(re.findall(r'[0-9]', every_pdf))
    
    final_list=extract_pdf_text(every_pdf)
    
    for i in range(len(final_list)):
        final_list[i][0] = check_name_in_list(final_list[i][0])
    
    votelist = final_list
    #save_votelist (votelist)
        
    global initial_df
    initial_df = pd.DataFrame(votelist)
    #initial_df = initial_df[initial_df[0].apply(lambda x: not x.startswith('ST'))]
    #initial_df = initial_df[initial_df[0].apply(lambda x: not x.startswith('Pierre Heusser'))]
    
    #initial_df[0] = initial_df[0].apply(lambda x: x.replace(' :', ''))
    #initial_df[0] = initial_df[0].apply(lambda x: x.replace(':', ''))
    #initial_df[0] = initial_df[0].apply(lambda x: x.replace(' nimmt Stellung', ''))
        
    df_with_length = add_length(initial_df)
    #save_vote_df(df_with_length)
    
    global name_ohne_partei
    name_ohne_partei =[]
    
    global df_in_progress
    df_in_progress = split_parties (df_with_length)
       
    search_list = build_searchlist (df_in_progress)
    return search_list

### 2.3 run

In [ ]:
# Schlaufe, welche eine Liste mit den PDFs abarbeitet. 
# Endresultat ist das df mit allen Voten sowie Infos zu deren Länge und den RednerInnen
for every_pdf in list_of_pdfs:
    search_list = part1 (every_pdf)
    resultlist = []
    for every_name in search_list:
        global searchname
        searchname = every_name[0]
        searchpartei = every_name[1]
        outp = membquest (searchname, searchpartei)
        resultlist.append(outp)
        
    nampi_dict_save()
    
    # Aufbau Resultate-df
    result_df = pd.DataFrame(resultlist)    
    result_df.drop(columns=[1,2,3], inplace=True)
    final_df = pd.concat([df_in_progress, result_df], axis=1)
    final_df = clean_final_df(final_df)
    move_df = final_df.copy(deep=True)
    
    # Prep to move to uber_df
    move_df.insert(4,'Sitzung','')
    move_df['Sitzung'] = sitzungs_id
    move_df.drop(['Votum'], axis=1, inplace=True)
    
    # !!! Index ist noch falsch, nicht fortlaufend
    uber_df = uber_df.append(move_df)
    #uber_df.reset_index()
    uber_df_save()
   